# 毕业论文`GRAD`
## 建立多元线性回归模型，定量评估影响

*`Evan`*\
*`2023-11-15`*
---

In [15]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics


import sys
sys.path.append('../../src/')
from namelist import *

# silence the warning note
import warnings
warnings.filterwarnings("ignore")

## 读取数据

In [17]:
lowyears  = [2014,2015,2016]
highyears = [2019,2021,2022]
datapath  = datadir + 'Contribution/data/'

df = {}
for year in lowyears:
    df[year] = pd.read_excel(datapath + f'SIM_PRD_Sep_{year}.xlsx',index_col=0)
dflow = pd.concat(df,axis=0)
df = {}
for year in highyears:
    df[year] = pd.read_excel(datapath + f'SIM_PRD_Sep_{year}.xlsx',index_col=0)
dfhigh = pd.concat(df,axis=0)

dflow.reset_index(level=0,inplace=True)
dflow.drop(columns='level_0',inplace=True)
dfhigh.reset_index(level=0,inplace=True)
dfhigh.drop(columns='level_0',inplace=True)

## 划分训练集和测试集

In [19]:
dflow.columns

Index(['SFC_TMP', 'SOL_RAD', 'RH', 'PRES', 'WSPD10', 'WDIR10', 'PBLH',
       'CloudFRAC', 'O3', 'NO2', 'VOC', 'PM25', 'ISOP'],
      dtype='object')

低值年

In [60]:
xlist = ['SFC_TMP', 'SOL_RAD', 'RH', 'PRES', 
         'WSPD10', 'WDIR10', 'PBLH','CloudFRAC',
         'NO2', 'VOC', 'PM25', 'ISOP']

In [29]:
X = dflow[xlist]
y = dflow['O3']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 初始化线性回归模型
model = LinearRegression()

# 在训练集上拟合模型
model.fit(X_train, y_train)

# 在测试集上进行预测
y_pred = model.predict(X_test)

# 评估模型性能
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

# 获取截距
intercept = model.intercept_
print('截距:', intercept)

# 获取各个自变量的系数
coefficients = model.coef_
print('系数:', coefficients)

equation = f'O3 = {intercept:.2f}'

for i in range(len(xlist)):
    equation += f' + {coefficients[i]:.2f}*{xlist[i]}'

print('多元线性回归方程为: '+equation)

Mean Absolute Error: 18.814984059185303
Mean Squared Error: 585.5589065522131
Root Mean Squared Error: 24.198324457536582
截距: 181.41360320046516
系数: [ 6.83274881e+00  3.28898214e-02 -1.05511319e+00 -2.01157007e-01
 -7.17076484e+00  1.19374927e-01 -1.15113707e-02  5.75421644e+01
 -1.75438095e+00 -6.08697246e-02  1.11796657e+00 -1.21759995e+00]
多元线性回归方程为: O3 = 181.41 + 6.83*SFC_TMP + 0.03*SOL_RAD + -1.06*RH + -0.20*PRES + -7.17*WSPD10 + 0.12*WDIR10 + -0.01*PBLH + 57.54*CloudFRAC + -1.75*NO2 + -0.06*VOC + 1.12*PM25 + -1.22*ISOP


高值年

In [30]:
xlist = ['SFC_TMP', 'SOL_RAD', 'RH', 'PRES', 
         'WSPD10', 'WDIR10', 'PBLH','CloudFRAC',
         'NO2', 'VOC', 'PM25', 'ISOP']

X = dfhigh[xlist]
y = dfhigh['O3']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 初始化线性回归模型
model = LinearRegression()

# 在训练集上拟合模型
model.fit(X_train, y_train)

# 在测试集上进行预测
y_pred = model.predict(X_test)

# 评估模型性能
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))
print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

# 获取截距
intercept = model.intercept_
print('截距:', intercept)

# 获取各个自变量的系数
coefficients = model.coef_
print('系数:', coefficients)

equation = f'O3 = {intercept:.2f}'

for i in range(len(xlist)):
    equation += f' + {coefficients[i]:.2f}*{xlist[i]}'

print('多元线性回归方程为: '+equation)

Mean Absolute Error: 17.439489172451708
Mean Squared Error: 514.670791365239
Root Mean Squared Error: 22.68635694344156
截距: -2152.735389526623
系数: [ 6.24354931e+00  2.61379950e-02 -1.36316647e-01  2.05452467e+00
  1.10376581e+00  1.75177572e-01 -4.17398756e-06  3.02126675e+01
 -1.89243646e+00  3.15854546e-01  9.53244919e-01 -6.47644523e-01]
多元线性回归方程为: O3 = -2152.74 + 6.24*SFC_TMP + 0.03*SOL_RAD + -0.14*RH + 2.05*PRES + 1.10*WSPD10 + 0.18*WDIR10 + -0.00*PBLH + 30.21*CloudFRAC + -1.89*NO2 + 0.32*VOC + 0.95*PM25 + -0.65*ISOP


另一种做多元回归的方法

In [40]:
import statsmodels.api as sm

X = dflow[xlist]
y = dflow['O3']
# 添加常数列
X = sm.add_constant(X)
# 进行多元线性逐步回归
model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                     O3   R-squared:                       0.763
Model:                            OLS   Adj. R-squared:                  0.762
Method:                 Least Squares   F-statistic:                     576.5
Date:                Wed, 15 Nov 2023   Prob (F-statistic):               0.00
Time:                        21:20:35   Log-Likelihood:                -9935.8
No. Observations:                2160   AIC:                         1.990e+04
Df Residuals:                    2147   BIC:                         1.997e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        327.3749    228.050      1.436      0.1

In [33]:
X = dfhigh[xlist]
y = dfhigh['O3']

X = sm.add_constant(X)

model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                     O3   R-squared:                       0.767
Model:                            OLS   Adj. R-squared:                  0.765
Method:                 Least Squares   F-statistic:                     588.1
Date:                Wed, 15 Nov 2023   Prob (F-statistic):               0.00
Time:                        21:05:21   Log-Likelihood:                -9802.9
No. Observations:                2160   AIC:                         1.963e+04
Df Residuals:                    2147   BIC:                         1.971e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -2026.0446    205.033     -9.882      0.0

## 多元线性逐步回归

In [51]:
import statsmodels.formula.api as smf
from patsy import dmatrices
import itertools as it
import random

target = 'O3'
variate = xlist
testdata = dflow

# 定义多个数组，用来分别用来添加变量，删除变量
x = []
variate_add = []
variate_del = variate.copy()
# print(variate_del)
y = random.sample(variate, 3)  # 随机生成一个选模型，3为变量的个数
print(y)
# 将随机生成的三个变量分别输入到 添加变量和删除变量的数组
for i in y:
    variate_add.append(i)
    x.append(i)
    variate_del.remove(i)

global aic  # 设置全局变量 这里选择AIC值作为指标
formula = "{}~{}".format(target, "+".join(variate_add))  # 将自变量名连接起来
aic = smf.ols(formula=formula, data=testdata).fit().aic  # 获取随机函数的AIC值，与后面的进行对比
print("随机化选模型为：{}~{}，对应的AIC值为：{}".format(target, "+".join(variate_add), aic))
print("\n")


# 添加变量
def forward():
    score_add = []
    global best_add_score
    global best_add_c
    print("添加变量")
    for c in variate_del:
        formula = "{}~{}".format(target, "+".join(variate_add + [c]))
        score = smf.ols(formula=formula, data=testdata).fit().aic
        score_add.append((score, c))  # 将添加的变量，以及新的AIC值一起存储在数组中
        print('自变量为{}，对应的AIC值为：{}'.format("+".join(variate_add + [c]), score))

    score_add.sort()  # 对数组内的数据进行排序，选择出AIC值最小的
    best_add_score, best_add_c = score_add.pop(0)
    print("最小AIC值为：{}".format(best_add_score))
    print("\n")


# 删除变量
def backward():
    score_del = []
    global best_del_score
    global best_del_c
    print("剔除变量")
    for i in x:
        select = x.copy()  # copy一个集合，避免重复修改到原集合
        select.remove(i)
        formula = "{}~{}".format(target, "+".join(select))
        score = smf.ols(formula=formula, data=testdata).fit().aic
        print('自变量为{}，对应的AIC值为：{}'.format("+".join(select), score))
        score_del.append((score, i))

    score_del.sort()  # 排序，方便将最小值输出
    best_del_score, best_del_c = score_del.pop(0)  # 将最小的AIC值以及对应剔除的变量分别赋值
    print("最小AIC值为：{}".format(best_del_score))
    print("\n")


print("剩余变量为：{}".format(variate_del))
forward()
backward()

while variate:
    if aic < best_add_score < best_del_score or aic < best_del_score < best_add_score:
        print("当前回归方程为最优回归方程，为{}~{}，AIC值为：{}".format(target, "+".join(variate_add), aic))
        break
    elif best_add_score < best_del_score < aic or best_add_score < aic < best_del_score:
        print("目前最小aic值为{}".format(best_add_score))
        print('选择自变量：{}'.format("+".join(variate_add + [best_add_c])))
        print('\n')
        variate_del.remove(best_add_c)
        variate_add.append(best_add_c)
        print("剩余变量为：{}".format(variate_del))
        aic = best_add_score
        forward()
    else:
        print('当前最小AIC值为：{}'.format(best_del_score))
        print('需要剔除的变量为：{}'.format(best_del_c))
        aic = best_del_score  # 将AIC值较小的选模型AIC值赋给aic再接着下一轮的对比
        x.remove(best_del_c)  # 在原集合上剔除选模型所对应剔除的变量
        backward()


['WDIR10', 'ISOP', 'SFC_TMP']
随机化选模型为：O3~WDIR10+ISOP+SFC_TMP，对应的AIC值为：21550.791022474325


剩余变量为：['SOL_RAD', 'RH', 'PRES', 'WSPD10', 'PBLH', 'CloudFRAC', 'NO2', 'VOC', 'PM25']
添加变量
自变量为WDIR10+ISOP+SFC_TMP+SOL_RAD，对应的AIC值为：21138.592833633367
自变量为WDIR10+ISOP+SFC_TMP+RH，对应的AIC值为：21163.29616077791
自变量为WDIR10+ISOP+SFC_TMP+PRES，对应的AIC值为：21530.069540732078
自变量为WDIR10+ISOP+SFC_TMP+WSPD10，对应的AIC值为：21552.702816814657
自变量为WDIR10+ISOP+SFC_TMP+PBLH，对应的AIC值为：21082.862312605455
自变量为WDIR10+ISOP+SFC_TMP+CloudFRAC，对应的AIC值为：21552.231139999465
自变量为WDIR10+ISOP+SFC_TMP+NO2，对应的AIC值为：20885.766418405543
自变量为WDIR10+ISOP+SFC_TMP+VOC，对应的AIC值为：21545.65945189967
自变量为WDIR10+ISOP+SFC_TMP+PM25，对应的AIC值为：21044.97842463813
最小AIC值为：20885.766418405543


剔除变量
自变量为ISOP+SFC_TMP，对应的AIC值为：21567.04214607352
自变量为WDIR10+SFC_TMP，对应的AIC值为：21773.08295731274
自变量为WDIR10+ISOP，对应的AIC值为：22020.703746293755
最小AIC值为：21567.04214607352


目前最小aic值为20885.766418405543
选择自变量：WDIR10+ISOP+SFC_TMP+NO2


剩余变量为：['SOL_RAD', 'RH', 'PRES', 'WSPD10', 'PBLH

选择逐步回归的变量建立线性回归

In [66]:
xlist = ['SFC_TMP', 'SOL_RAD', 'RH', 'PRES', 
         'WSPD10', 'WDIR10', 'PBLH','CloudFRAC',
         'NO2', 'VOC', 'PM25', 'ISOP']
xlist.remove('VOC')
xlist.remove('PRES')

In [67]:
X = dflow[xlist]
y = dflow['O3']
# 添加常数列
X = sm.add_constant(X)
# 进行多元线性逐步回归
model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                     O3   R-squared:                       0.763
Model:                            OLS   Adj. R-squared:                  0.762
Method:                 Least Squares   F-statistic:                     691.0
Date:                Wed, 15 Nov 2023   Prob (F-statistic):               0.00
Time:                        21:43:41   Log-Likelihood:                -9937.6
No. Observations:                2160   AIC:                         1.990e+04
Df Residuals:                    2149   BIC:                         1.996e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -29.3599     14.460     -2.030      0.0

In [68]:
X = dfhigh[xlist]
y = dfhigh['O3']
# 添加常数列
X = sm.add_constant(X)
# 进行多元线性逐步回归
model = sm.OLS(y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                     O3   R-squared:                       0.756
Model:                            OLS   Adj. R-squared:                  0.755
Method:                 Least Squares   F-statistic:                     665.4
Date:                Wed, 15 Nov 2023   Prob (F-statistic):               0.00
Time:                        21:44:33   Log-Likelihood:                -9852.0
No. Observations:                2160   AIC:                         1.973e+04
Df Residuals:                    2149   BIC:                         1.979e+04
Df Model:                          10                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        -31.6128     11.107     -2.846      0.0